In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import librosa
import numpy as np
import os
from glob import glob
import json

In [4]:
audio_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix"
json_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"
cqt_out_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt"

In [1]:
def extract_harmonic_cqt(audio_path, sr=22050, hop_length=512, save_path=None):
    y, _ = librosa.load(audio_path, sr=sr)

    # 🎵 Referans frekans: E2 (yaklaşık 82.41 Hz)
    f0 = librosa.note_to_hz('E2')

    # 🔁 Harmonik oranları: 1 subharmonic + 5 harmonik
    ratios = [0.5, 1, 2, 3, 4, 5]
    cqts = []

    for r in ratios:
        cqt = librosa.cqt(y=y,
                          sr=sr,
                          hop_length=hop_length,
                          fmin=f0 * r,
                          n_bins=3 * 12 * 4,  # 4 oktav, 3 bin/semitone
                          bins_per_octave=36)
        cqt = np.abs(cqt)
        cqts.append(cqt)

    # 🔢 Shape: (6, time, freq)
    cqts = np.stack(cqts, axis=0)
    cqts = cqts.transpose(0, 2, 1)  # (6, time, freq)

    if save_path:
        np.save(save_path, cqts)
        print(f"✅ Harmonic CQT saved: {save_path}")

    return cqts

In [5]:
audio_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_mic"
save_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt"

cqt = extract_harmonic_cqt(audio_path, save_path=save_path)
print("CQT shape:", cqt.shape)  # 🔹 (6, T, 144)

✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_BN1-129-Eb_comp_mic.wav
CQT shape: (6, 962, 144)


In [7]:
wav_files = glob(os.path.join(audio_dir, "*.wav"))

for wav_path in wav_files:
    base = os.path.basename(wav_path).replace(".wav", "")
    save_name = base + "_cqt.npy"  # istediğin format
    save_path = os.path.join(cqt_out_dir, save_name)
    if not os.path.exists(save_path):
        extract_harmonic_cqt(wav_path, save_path=save_path)
    else:
        print(f"✅ Already exists: {save_path}")

✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_Jazz2-110-Bb_comp_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_BN2-131-B_solo_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_Funk2-119-G_solo_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_Jazz1-130-D_comp_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_Jazz1-200-B_comp_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_BN3-154-E_comp_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_Jazz1-200-B_solo_mix_cqt.npy
✅ Harmonic CQT saved: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/G

In [8]:
import numpy as np

cqt_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_BN1-129-Eb_solo_mix_cqt.npy"

cqt = np.load(cqt_path)
print("CQT shape:", cqt.shape)
print("Min:", np.min(cqt), "Max:", np.max(cqt), "Mean:", np.mean(cqt))

CQT shape: (6, 962, 144)
Min: 1.2202663e-06 Max: 8.947252 Mean: 0.07137844


In [10]:
import json

json_path = cqt_path.replace("/cqt/", "/merge_annotation/").replace("_cqt.npy", "_fretnet.json")

with open(json_path, "r") as f:
    data = json.load(f)

times = np.array(data["times"])
print("Frame count in CQT:", cqt.shape[1])
print("Frame count in times:", len(times))

Frame count in CQT: 962
Frame count in times: 1044


In [11]:
import os
import numpy as np
import json
import pandas as pd

cqt_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt"
json_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"

mismatches = []
differences = []

for fname in sorted(os.listdir(cqt_dir)):
    if not fname.endswith(".npy"):
        continue

    cqt_path = os.path.join(cqt_dir, fname)
    json_name = fname.replace("_cqt.npy", "_fretnet.json")
    json_path = os.path.join(json_dir, json_name)

    if not os.path.exists(json_path):
        mismatches.append((fname, "MISSING_JSON", "-"))
        continue

    try:
        cqt = np.load(cqt_path)
        with open(json_path, "r") as f:
            data = json.load(f)

        times = data["times"]
        cqt_len = cqt.shape[1]
        times_len = len(times)
        diff = cqt_len - times_len
        differences.append(diff)

        if diff != 0:
            mismatches.append((fname, cqt_len, times_len))
    except Exception as e:
        mismatches.append((fname, "ERROR", str(e)))

df = pd.DataFrame(mismatches, columns=["Filename", "CQT_frames", "Times_frames"])
print("📊 Uyuşmayan dosya sayısı:", len(df))
print("🧮 En küçük fark:", min(differences) if differences else "-")
print("🧮 En büyük fark:", max(differences) if differences else "-")

display(df)

📊 Uyuşmayan dosya sayısı: 360
🧮 En küçük fark: -173
🧮 En büyük fark: -54


,Filename,CQT_frames,Times_frames
0,00_BN1-129-Eb_comp_mix_cqt.npy,962,1044
1,00_BN1-129-Eb_solo_mix_cqt.npy,962,1044
2,00_BN1-147-Gb_comp_mix_cqt.npy,844,918
3,00_BN1-147-Gb_solo_mix_cqt.npy,844,918
4,00_BN2-131-B_comp_mix_cqt.npy,1263,1368
...,...,...,...
355,05_SS2-88-F_solo_mix_cqt.npy,1879,2043
356,05_SS3-84-Bb_comp_mix_cqt.npy,1969,2142
357,05_SS3-84-Bb_solo_mix_cqt.npy,1969,2142
358,05_SS3-98-C_comp_mix_cqt.npy,1688,1836


CQT verisini her model çekerken kırpacak